In [1]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os, glob
import sys
import matplotlib
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4)
import random
%matplotlib inline
import pretty_midi as pm
import tensorflow as tf
from tqdm import tqdm

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
#Literally load all midis as prettyMidis into a list
if(os.getcwd().split('/')[-1] != 'unpackedMidis'):
    os.chdir("unpackedMidis")

fileList = [x for x in glob.glob("*.mid")]
songList = []
badSongs = []
#doing all 16k crashes my whole computer. This may have something to do with Intel processors "leaking" in the most
#recent ubuntu dist. You should look that up
for file in tqdm(range(1000)):
    try:
        songList.append(pm.PrettyMIDI(fileList[file]))
    except:
        badSongs.append(fileList[file])

  3%|▎         | 27/1000 [00:11<07:08,  2.27it/s]/home/javi/.local/lib/python3.6/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)
100%|██████████| 1000/1000 [06:03<00:00,  2.75it/s]


In [11]:
songList[0].instruments[0].notes[0]

Note(start=4.528200, end=4.590109, pitch=42, velocity=115)

In [ ]:
#So this builds two dictionaries. One for notes -> numbers, and the other for numbers -> notes
#You could also use pretty_midis pm.key_name_to_key_number() and pm.key_number_to_key_name()
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

#This is the actual LSTM model
def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # n-layer LSTM with n_hidden units.
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']
